In [12]:
using Pkg
Pkg.activate(@__DIR__);
pkg"instantiate"
pkg"precompile"

using RigidBodyDynamics
using MeshCatMechanisms
using StaticArrays
using RigidBodyDynamics.Contact
using RigidBodySim

  Activating environment at `D:\Documents\git_workspace\flyhopper\scripts\Julia\Project.toml`


In [13]:
curdir = pwd()
const URDFPATH = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
bot = parse_urdf(URDFPATH, floating=true)
remove_fixed_tree_joints!(bot);

In [14]:
# add environment
rootframe = root_frame(bot)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1))
add_environment_primitive!(bot, ground);

In [15]:
g = -9.81 # gravitational acceleration in z-direction
state = MechanismState(bot)
set_configuration!(state, [1; 0; 0; 0; 0; 0; 0.7])
q = configuration(state)
v = velocity(state)

8-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [34]:
# build_contact_points(bot)[1]
foot = bodies(bot)[2]
Nmdl = RigidBodyDynamics.Contact.hunt_crossley_hertz()   
Fmdl = RigidBodyDynamics.Contact.ViscoelasticCoulombModel(0.3, 30e3,0.3)   
scm = RigidBodyDynamics.Contact.SoftContactModel(Nmdl, Fmdl)
# point = convert(Point3D{SVector{3, Float64}}, Point3D(default_frame(findbody(bot, "base_link")), 0, 0, 0))
point = convert(Point3D{SVector{3, Float64}}, Point3D(default_frame(foot), 0, 0, 0))
cp =  RigidBodyDynamics.Contact.ContactPoint(point, scm)

# add_contact_point!(findbody(bot, "Link 2"), cp)
add_contact_point!(foot, cp)
# contact_points(findbody(bot, "Link 2"))

In [35]:
frame = default_frame(bodies(bot)[1])
point = Point3D(frame, 0.0, 0.0, 0.0)
normal = FreeVector3D(frame, 0., 0., 1.)
halfspace = RigidBodyDynamics.Contact.HalfSpace3D(point, normal)

push!(bot.environment.halfspaces, halfspace)

3-element Vector{HalfSpace3D{Float64}}:
 HalfSpace3D{Float64}(Point3D in "world": [0.0, 0.0, 0.0], FreeVector3D in "world": [0.0, 0.0, 1.0])
 HalfSpace3D{Float64}(Point3D in "world": [0.0, 0.0, 0.0], FreeVector3D in "world": [0.0, 0.0, 1.0])
 HalfSpace3D{Float64}(Point3D in "world": [0.0, 0.0, 0.0], FreeVector3D in "world": [0.0, 0.0, 1.0])

In [36]:
center_of_mass(state)

Point3D in "world": [-0.002547131739826878, -3.3470181064453804e-8, 0.697058817292039]

In [37]:
q0, q1 = joints(bot)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    # velocity_range: Return the range of indices into the joint velocity vector v corresponding to joint "joint"
    torques[velocity_range(state, q0)] .= -1 .* velocity(state, q0)
    torques[velocity_range(state, q1)] .= -1 .* velocity(state, q1)
end;

In [38]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

LoadError: BoundsError: attempt to access 0-element Vector{Vector{DefaultSoftContactState{Float64}}} at index [1]

In [10]:
zero_velocity!(state)
# set_configuration!(state, q0, 0.7)
# set_configuration!(state, q1, -0.8);
mvis = MechanismVisualizer(bot, URDFVisuals(URDFPATH));

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8700
└ @ MeshCat C:\Users\bboks\.julia\packages\MeshCat\X2AUA\src\visualizer.jl:73


In [11]:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);